## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import csv
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def find_delimiter(path):
    sniffer = csv.Sniffer()
    with open(path) as fp:
        delimiter = sniffer.sniff(fp.read(5000)).delimiter
    return delimiter

In [ ]:
deli = find_delimiter('/content/drive/MyDrive/MMO/women-clothing-accessories.csv')
df = pd.read_csv('/content/drive/MyDrive/MMO/women-clothing-accessories.csv', delimiter = deli)

In [ ]:
df

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.feature_extraction.text import CountVectorizer 
import string 
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import * 
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 8.2 MB 51.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=22f0f1b62584a6437f582ae06026149c3ff6d584ea10de1482a8cd49902768db
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
punkt = string.punctuation 
noise = stopwords.words("russian")

## **Токенизация + нижний регистер**

In [ ]:
p = df['review']

for i in range(len(df)):

    p_low = p[i].lower()
    l = word_tokenize(p_low) 
    p[i] = l

df['review'] = p 

In [ ]:
df

,review,sentiment
0,"[качество, плохое, пошив, ужасный, (, горловин...",negative
1,"[товар, отдали, другому, человеку, ,, я, не, п...",negative
2,"[ужасная, синтетика, !, тонкая, ,, ничего, общ...",negative
3,"[товар, не, пришел, ,, продавец, продлил, защи...",negative
4,"[кофточка, голая, синтетика, ,, носить, не, во...",negative
...,...,...
89995,"[сделано, достаточно, хорошо, ., на, ткани, сд...",positive
89996,"[накидка, шикарная, ., спасибо, большое, провд...",positive
89997,"[спасибо, большое, ), продовца, рекомендую, .....",positive
89998,"[очень, довольна, заказом, !, меньше, месяца, ...",positive


## **Лемматизация**

**Поменял лемматизацию и удаление слов местами**

In [ ]:
pymorphy2_analyzer = MorphAnalyzer() 

df["review"] = df['review'].apply(lambda row: [pymorphy2_analyzer.parse(i)[0].normal_form for i in row if i])

In [ ]:
df

,review,sentiment
0,"[качество, плохой, пошив, ужасный, (, горловин...",negative
1,"[товар, отдать, другой, человек, ,, я, не, пол...",negative
2,"[ужасный, синтетик, !, тонкий, ,, ничего, общи...",negative
3,"[товар, не, прийти, ,, продавец, продлить, защ...",negative
4,"[кофточка, голый, синтетик, ,, носить, не, воз...",negative
...,...,...
89995,"[сделать, достаточно, хорошо, ., на, ткань, сд...",positive
89996,"[накидка, шикарный, ., спасибо, большой, провд...",positive
89997,"[спасибо, большой, ), продовца, рекомендовать,...",positive
89998,"[очень, довольный, заказ, !, маленький, месяц,...",positive


##**Удаление стоп-слов и пунктуации**

In [ ]:
df["review"] = df['review'].apply(lambda row: [i for i in row if i not in noise and i not in punkt])

In [ ]:
df

,review,sentiment
0,"[качество, плохой, пошив, ужасный, горловина, ...",negative
1,"[товар, отдать, человек, получить, посылка, ла...",negative
2,"[ужасный, синтетик, тонкий, общий, представить...",negative
3,"[товар, прийти, продавец, продлить, защита, со...",negative
4,"[кофточка, голый, синтетик, носить, возможно]",negative
...,...,...
89995,"[сделать, достаточно, ткань, сделать, рисунок,...",positive
89996,"[накидка, шикарный, спасибо, большой, провдо, ...",positive
89997,"[спасибо, большой, продовца, рекомендовать, .....",positive
89998,"[очень, довольный, заказ, маленький, месяц, рб...",positive


In [ ]:
for i in range(len(df)): # делаем str из list
    df['review'][i] = " ".join(df['review'][i])

In [ ]:
df

,review,sentiment
0,качество плохой пошив ужасный горловина напере...,negative
1,товар отдать человек получить посылка ладный д...,negative
2,ужасный синтетик тонкий общий представить карт...,negative
3,товар прийти продавец продлить защита согласие...,negative
4,кофточка голый синтетик носить возможно,negative
...,...,...
89995,сделать достаточно ткань сделать рисунок замет...,positive
89996,накидка шикарный спасибо большой провдо линять...,positive
89997,спасибо большой продовца рекомендовать .. зака...,positive
89998,очень довольный заказ маленький месяц рб курье...,positive


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size = 0.2)

In [ ]:
x_train

43494    очень странно сидеть рост 170 кроткий длина по...
23862                                                   ..
28341    самый куртка супер качество 2xl =44-46 продаве...
9260     любимый магазин разочаровать купон 11.11 видим...
5165     оказаться мало размер блузка отличный состояни...
                               ...                        
81619    скажем качество 4 ........ доставить нитка тор...
40471    поверхность куртка мягкий приятный наощупь либ...
40428                     дырка карман везде торчать нитка
15168    чашечка соответствовать размер третий размер м...
50583    ращмера подойти пахнуть плотный рукав вообще о...
Name: review, Length: 72000, dtype: object

## **Мешок n-грамм**

**Исправил парметры**

In [ ]:
ngram_range = [(1, 2), (1, 3), (1, 4)]

In [ ]:
acc = -100

for ngram in range(len(ngram_range)): # подбор параметров

    vectorizer = CountVectorizer(ngram_range=ngram_range[ngram])

    vectorized_x_train = vectorizer.fit_transform(x_train)
    vectorized_x_test = vectorizer.transform(x_test)


    clf = MultinomialNB()
    clf.fit(vectorized_x_train, y_train)


    pred = clf.predict(vectorized_x_test)

    l = accuracy_score(y_test, pred)

    if l > acc:

        acc = l

        best_ngram_range = ngram_range[ngram]


print(acc)
print(best_ngram_range)

0.7173888888888889
(1, 4)


In [ ]:
vectorizer = CountVectorizer(ngram_range=best_ngram_range)
vectorized_x_train = vectorizer.fit_transform(x_train)
vectorized_x_test = vectorizer.transform(x_test)

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB()

In [ ]:
pred_v = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.60      0.63      0.62      5899
    negative       0.72      0.66      0.69      6097
    positive       0.83      0.86      0.84      6004

    accuracy                           0.72     18000
   macro avg       0.72      0.72      0.72     18000
weighted avg       0.72      0.72      0.72     18000



## **tf-idf**

In [ ]:
from sklearn.metrics import accuracy_score

**Изменил подбор гиперпараметров**

In [ ]:
max_df = [0.25, 0.5]
max_features = [1000, 10000, 50000]
min_df = [0, 0.2]
ngram_range = [(1, 2), (1, 3)]

In [ ]:
acc = -1

for max_i in range(len(max_df)): # подбор параметров

    for max_f in range(len(max_features)):

        for min_i in range(len(min_df)):

            for ngram in range(len(ngram_range)):


                tfidf_vectorizer = TfidfVectorizer(max_df = max_df[max_i], max_features = max_features[max_f], min_df = min_df[min_i], ngram_range = ngram_range[ngram])
        
                tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)
                tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)


                clf = MultinomialNB()
                clf.fit(tfidf_vectorized_x_train, y_train)


                pred = clf.predict(tfidf_vectorized_x_test)

                l = accuracy_score(y_test, pred)

                if l > acc:

                    acc = l

                    best_max_df, best_max_features, best_min_df, best_ngram_range = max_df[max_i], max_features[max_f], min_df[min_i], ngram_range[ngram]




print(acc)
print(best_max_df, best_max_features, best_min_df, best_ngram_range)

0.7173333333333334
0.25 50000 0 (1, 2)


In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df = best_max_df, max_features = best_max_features, min_df = best_min_df, ngram_range = best_ngram_range)
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

In [ ]:
clf = MultinomialNB()
clf.fit(tfidf_vectorized_x_train, y_train)

MultinomialNB()

In [ ]:
pred_t = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred_t))

              precision    recall  f1-score   support

    neautral       0.59      0.69      0.63      5899
    negative       0.74      0.62      0.67      6097
    positive       0.85      0.85      0.85      6004

    accuracy                           0.72     18000
   macro avg       0.72      0.72      0.72     18000
weighted avg       0.73      0.72      0.72     18000



## **Cимвольные n-граммы**

**Исправил гиперпараметры, теперь подбирвется только значение n**

In [ ]:
ngram_range = [(1, 3), (3, 6), (4, 7)]

In [ ]:
acc = -1

for ngram in range(len(ngram_range)):


    char_vectorizer = CountVectorizer(analyzer='char', ngram_range = ngram_range[ngram])
    char_vectorized_x_train = char_vectorizer.fit_transform(x_train)
    char_vectorized_x_test = char_vectorizer.transform(x_test)

    clf = MultinomialNB()
    clf.fit(char_vectorized_x_train, y_train)

    pred = clf.predict(char_vectorized_x_test)

    l = accuracy_score(y_test, pred)

    if l > acc:

        acc = l

        best_ngram_range = ngram_range[ngram]




print(acc)
print(best_ngram_range)

0.7056111111111111
(4, 7)


In [ ]:
char_vectorizer = CountVectorizer(analyzer='char', ngram_range = best_ngram_range)

# обучаем его и сразу применяем к x_train
char_vectorized_x_train = char_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = MultinomialNB()
clf.fit(char_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
char_vectorized_x_test = char_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred_c = clf.predict(char_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.57      0.68      0.62      5899
    negative       0.73      0.60      0.66      6097
    positive       0.84      0.83      0.84      6004

    accuracy                           0.71     18000
   macro avg       0.71      0.71      0.71     18000
weighted avg       0.72      0.71      0.71     18000



## **ИТОГ**

In [ ]:
result_v = pd.Series([precision_score(y_test, pred_v, average = 'macro'), 
                      recall_score(y_test, pred_v, average = 'macro'), 
                      f1_score(y_test, pred_v, average = 'macro'), 
                      accuracy_score(y_test, pred_v)], 
                     index=['precision','recall','f1-score','accuracy'], name ='Мешок n-грамм')

In [ ]:
result_t = pd.Series([precision_score(y_test, pred_t, average = 'macro'), 
                      recall_score(y_test, pred_t, average = 'macro'), 
                      f1_score(y_test, pred_t, average = 'macro'), 
                      accuracy_score(y_test, pred_t)], 
                     index=['precision','recall','f1-score','accuracy'], name = 'TF-IDF')

In [ ]:
result_c = pd.Series([precision_score(y_test, pred_c, average = 'macro'), 
                      recall_score(y_test, pred_c, average = 'macro'), 
                      f1_score(y_test, pred_c, average = 'macro'), 
                      accuracy_score(y_test, pred_c)], 
                     index=['precision','recall','f1-score','accuracy'], name = 'Сим. n-граммы')

In [ ]:
pd.concat([result_v, result_t, result_c], axis =1)

,Мешок n-грамм,TF-IDF,Сим. n-граммы
precision,0.717441,0.724880,0.714964
recall,0.717214,0.717682,0.706017
f1-score,0.716542,0.718091,0.706790
accuracy,0.717389,0.717333,0.705611


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
reg_state = r'\b\w.'
re.findall(reg_state, 'sklearn pandas numpy nltk string')

['sk', 'pa', 'nu', 'nl', 'st']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
sen = 'Скоро пойду спать. Завтра сложный день. Много дел. скорее бы отпуск. Хочу уехать на юг'
reg = r'[.]'
re.split(reg, sen, maxsplit = 3)

['Скоро пойду спать',
 ' Завтра сложный день',
 ' Много дел',
 ' скорее бы отпуск. Хочу уехать на юг']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
sens = '1703 год основания СПБ'
re.sub('\d', 'DIG', sens)

'DIGDIGDIGDIG год основания СПБ'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
sent = 'https://garantex.io/trading/usdtrub данная ссылка позволит торговать криптовалютой'
re.sub(r'http\S*.', '', sent)

'данная ссылка позволит торговать криптовалютой'

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('\w\w[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
domens = re.compile(r'@[\w-]+\.[\w.-]+')
domens.findall('abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']